This is the script I used to find the words from alignments.
I put it here for completeness, but it is not necessary to run it to get the results.

In [3]:
from pathlib import Path

import torchaudio
import pandas as pd
from textgrid import TextGrid

from ipywidgets import interact
from IPython.display import Audio, display

In [34]:
subsets = ['dev-clean', 'dev-other', 'test-clean', 'test-other']

In [15]:
dfs = []
for subset in subsets:
    alignments_dir = Path(f"/home/nicolvisser/Data/LibriSpeech/alignments/montreal-forced-aligner/english_us_mfa/textgrids/{subset}")
    waveforms_dir = Path(f"/home/nicolvisser/Datasets/LibriSpeech/{subset}")
    rows = []
    for alignment_path in alignments_dir.rglob("*.TextGrid"):
        tg = TextGrid.fromFile(alignment_path)
        speaker_id = alignment_path.parent.parent.name
        chapter_id = alignment_path.parent.name
        rel_path = alignment_path.relative_to(alignments_dir)
        wav_path = waveforms_dir / rel_path.with_suffix(".flac")
        for tier in tg.tiers:
            if tier.name == "words":
                for interval in tier.intervals:
                    if interval.mark != "":
                        rows.append(
                            {
                                "word": interval.mark,
                                "start": interval.minTime,
                                "end": interval.maxTime,
                                "speaker_id": speaker_id,
                                "chapter_id": chapter_id,
                                "filename": alignment_path.stem,
                                "wav_path": wav_path,
                            }
                        )
    df = pd.DataFrame(rows)
    df.eval("duration = end - start", inplace=True)
    df.eval("num_chars = word.str.len()", inplace=True)

    dfs.append(df)


Filter out words
- with less than 5 characters
- with less than 0.5 seconds duration
- that only occur once in the subset

In [25]:
for i in range(len(dfs)):
    dfs[i] = dfs[i].query("num_chars >= 5")
    dfs[i] = dfs[i].query("duration >= 0.5")
    dfs[i] = dfs[i].groupby("word").filter(lambda x: len(x) > 1)

    dfs[i] = dfs[i].sort_values(["word", "speaker_id"])
    dfs[i] = dfs[i].reset_index(drop=True)


In [31]:
dfs[0].groupby("word")['filename'].count().sort_values(ascending=False).head(5)

word
himself       37
themselves    23
myself        16
silence       14
things        14
Name: filename, dtype: int64

In [32]:
dfs[0].groupby("word")['filename'].count().sort_values(ascending=True).head(5)

word
intervals    2
glory        2
savagely     2
sauterne     2
satisfy      2
Name: filename, dtype: int64

In [33]:
@interact(i=(0, len(dfs[0])-1))
def get_word(i=0):
    word = dfs[0].iloc[i].word
    print(word)
    wav_path = dfs[0].iloc[i].wav_path
    wav, sr = torchaudio.load(wav_path)
    start = int(dfs[0].iloc[i].start*sr)
    end = int(dfs[0].iloc[i].end*sr)
    display(Audio(wav[:, start:end], rate=sr))

interactive(children=(IntSlider(value=0, description='i', max=4166), Output()), _dom_classes=('widget-interact…

In [35]:
for subset, df in zip(subsets, dfs):
    df[["word", "speaker_id", "filename", "start", "end"]].to_csv(f"{subset}.csv", index=True)